# Parallelizing Pandas DataFrame Apply

## Parallelization Code

In [1]:
import pandas as pd
import concurrent.futures as fut

In [2]:
def apply_df(df, *args, **kwargs):
    return df.apply(*args, **kwargs)

In [3]:
def apply_chunks(df, func, *args, executor=None, chunksize=10000, **kwargs):
    df_chunks = (df[i:i+chunksize] for i in range(0, df.shape[0], chunksize))
    
    if executor is None:
        # generate ad-hoc executor
        with fut.ProcessPoolExecutor() as ex:
            futures = [ex.submit(apply_df, df_part, func, *args, axis=1, **kwargs)
              for df_part in df_chunks]
            out = pd.concat([future.result() for future in futures])
    else:
        futures = [executor.submit(apply_df, df_part, func, *args, axis=1, **kwargs)
              for df_part in df_chunks]
        out = pd.concat([future.result() for future in futures])
    return out

Limitations:
* only works for axis=1
* does not work for lambda functions (they are not pickable) - one needs to define standard (non-anonymous) functions (via def).

## Test Case

In [4]:
import numpy as np

In [5]:
df = pd.DataFrame(np.random.randn(1000000, 5))
df.head()

,0,1,2,3,4
0,-0.623533,-0.023109,1.328919,-0.150032,0.110899
1,0.752936,0.673337,0.299617,-0.085345,-0.465252
2,-0.273059,0.008448,0.833176,-0.588263,0.771937
3,1.467874,0.789163,-0.421545,0.158145,0.832988
4,-0.928144,0.792610,-0.066542,1.977496,0.067713


In [6]:
def func(x):
    return sum(x)

### Standard Version

In [7]:
%time df['y'] = apply_df(df, func, axis=1)

CPU times: user 48.9 s, sys: 56 ms, total: 49 s
Wall time: 49 s


In [8]:
df.head()

,0,1,2,3,4,y
0,-0.623533,-0.023109,1.328919,-0.150032,0.110899,0.643145
1,0.752936,0.673337,0.299617,-0.085345,-0.465252,1.175292
2,-0.273059,0.008448,0.833176,-0.588263,0.771937,0.752238
3,1.467874,0.789163,-0.421545,0.158145,0.832988,2.826626
4,-0.928144,0.792610,-0.066542,1.977496,0.067713,1.843132


In [9]:
df.drop(columns='y', inplace=True)

### Parallel Version

In [10]:
%time df['y'] = apply_chunks(df, func)

CPU times: user 333 ms, sys: 99 ms, total: 432 ms
Wall time: 13.2 s


In [11]:
df.head()

,0,1,2,3,4,y
0,-0.623533,-0.023109,1.328919,-0.150032,0.110899,0.643145
1,0.752936,0.673337,0.299617,-0.085345,-0.465252,1.175292
2,-0.273059,0.008448,0.833176,-0.588263,0.771937,0.752238
3,1.467874,0.789163,-0.421545,0.158145,0.832988,2.826626
4,-0.928144,0.792610,-0.066542,1.977496,0.067713,1.843132


In [12]:
df.drop(columns='y', inplace=True)

### Dask

In [13]:
import dask.distributed

In [14]:
client = dask.distributed.Client()
client

Client Scheduler: tcp://127.0.0.1:38596 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 10.28 GB


In [15]:
%time df['y'] = apply_chunks(df, func, executor=client)

CPU times: user 3.88 s, sys: 587 ms, total: 4.47 s
Wall time: 17.1 s


In [16]:
df.head()

,0,1,2,3,4,y
0,-0.623533,-0.023109,1.328919,-0.150032,0.110899,0.643145
1,0.752936,0.673337,0.299617,-0.085345,-0.465252,1.175292
2,-0.273059,0.008448,0.833176,-0.588263,0.771937,0.752238
3,1.467874,0.789163,-0.421545,0.158145,0.832988,2.826626
4,-0.928144,0.792610,-0.066542,1.977496,0.067713,1.843132


In [17]:
df.drop(columns='y', inplace=True)

In [18]:
client.close()

### Vectorized Version

Just for comparison in this simple example.

Note that the use case for apply are just these cases where vectorization is not possible.

In [19]:
%time df['y'] = df.sum(axis=1)

CPU times: user 108 ms, sys: 30 ms, total: 138 ms
Wall time: 135 ms


In [20]:
df.head()

,0,1,2,3,4,y
0,-0.623533,-0.023109,1.328919,-0.150032,0.110899,0.643145
1,0.752936,0.673337,0.299617,-0.085345,-0.465252,1.175292
2,-0.273059,0.008448,0.833176,-0.588263,0.771937,0.752238
3,1.467874,0.789163,-0.421545,0.158145,0.832988,2.826626
4,-0.928144,0.792610,-0.066542,1.977496,0.067713,1.843132


In [21]:
df.drop(columns='y', inplace=True)

## Summary

A significant speed-up (factor 3 with 4 CPUs) was gained using Multiprocessing for Pandas DataFrame applys.

The speed gain using Dask was smaller, maybe it has more overhead.

No parallelization comes even close to (single CPU) vectorized calculations, but they are not possible in all cases.